# AAM Demo Script

In this demo script, we first introduce the AAM calculation pipeline step by step in tutorial style. It is a compliment to the paper and is provided to ensure reproducibility of the results. 

## Initialize and load data

The dataset used here is described in the paper. It can be replaced by your own data, with only a few slight modifications to the code. 

In [ ]:
import numpy as np
import scipy.io as sio
import scipy
import matplotlib.pyplot as plt
import math
%reload_ext autoreload
%autoreload 2

# Choose camera
IMG_SET_ID = 4 # Canon EF 50mm f/2.5
#IMG_SET_ID = 7 # Canon EF 50mm f/1.8 I

# Load data
path = "../data/"
dist_name = path + "distances_" + str(IMG_SET_ID) + ".mat"
PSF_name  = path + "GaussStd2Color_" + str(IMG_SET_ID) + ".mat"
PSF_NIR_name = path + "GaussStd2Nir_" + str(IMG_SET_ID) + ".mat"

mat_dist = sio.loadmat(dist_name)
mat_PSF = sio.loadmat(PSF_name)
mat_PSF_NIR = sio.loadmat(PSF_NIR_name)

distances = mat_dist['distancesCol']
PSF       = mat_PSF['GaussStd2Color']
PSF_NIR   = np.squeeze(mat_PSF_NIR['GaussStd2Nir'])
PSF[:,3] = PSF_NIR

assert distances.shape[0] == PSF.shape[0]
assert distances.shape[0] == PSF_NIR.shape[0]
print('loaded experimental data from {} distances and {} channels.'.format(*PSF.shape))

## Resample data uniformly. 

For the polynomial fitting to work correctly, we need to resample the experimental, non-uniform data of our camera with IMG_SET_ID 4. Two methods are proposed, an automatic method and a manual method. The results are shown below. In practice, we use the manual method , since it is close to the actual measurements (no interpolation required) and thus more accurate. 

In [ ]:
from aam import make_uniform
    
plt.plot(distances, PSF, '*') 
plt.ylabel('Raw Experimental PSF')
plt.show()

distances_uniform, PSF_uniform = make_uniform(distances, PSF, 'uniform')

plt.figure()
plt.plot(distances_uniform, PSF_uniform, '*')
plt.ylabel('Interpolated PSF')
plt.show()

if IMG_SET_ID == 4:
    distances_uniform, PSF_uniform = make_uniform(distances, PSF, 'manual')

    plt.figure()
    plt.plot(distances_uniform, PSF_uniform, '*')
    plt.ylabel('Manually resampled PSF')
    plt.show()

# No resampling is required for camera 7, since the data is 
# already approximately uniform.
if IMG_SET_ID == 7:
    distances_uniform = distances
    PSF_uniform = PSF

## Polynomial fitting

In order to simplify the calculation of the error metric, we fit a polynomial of degree 5 to the PSF curves. The below two cells visualize the results without noise (first cell) and with noise (second cell), proving the robustness of the polynomial fitting to noise. 

In [ ]:
from aam import polynomial_fitting, get_focus_distances

def plot_fitting(polyParams, x0, PSF):
    plt.figure()
    for i in range(polyParams.shape[0]):
        f = np.poly1d(polyParams[i])
        p = plt.plot(x, f(x) , '-')
        color = p[0].get_color()
        plt.plot(x, PSF[:, i], '*', color=color)
        plt.vlines(x0[i],ymin=0, ymax=60, color=color)
    plt.xlabel('distance [m]')
    plt.ylabel('PSF original and resampled')
    plt.show()

scaling = 1000
x = np.squeeze(distances_uniform/scaling)

polyParams = polynomial_fitting(x, PSF_uniform, degree=5)
x0 = get_focus_distances(polyParams, bounds=(x[0], x[-1]))
plot_fitting(polyParams, x0, PSF_uniform)

In [ ]:
# polynomial fitting with added noise. 
PSF_uniform_noisy = PSF_uniform.copy()
sigma_noise = 1e-3
PSF_uniform_noisy += np.random.normal(scale=sigma_noise*PSF_uniform, size=PSF_uniform.shape)
polyParams_noisy = polynomial_fitting(x, PSF_uniform_noisy, degree=5)
x0_noisy = get_focus_distances(polyParams_noisy, bounds=(x[0], x[-1]))
plot_fitting(polyParams_noisy, x0_noisy, PSF_uniform_noisy)

## Compute AAM

Given the polynomial fittings of all PSF curves, we can compute the AAM. 

In [ ]:
from aam import compute_aam

num_colors = 4
num_alphas = 51
alphaList = np.linspace(0.2, 0.5, num_alphas)
AAM = compute_aam(polyParams[:num_colors, :], x0[:num_colors], alphaList)

plt.plot(alphaList, AAM)
plt.xlabel('alpha'); plt.ylabel('AAM');
plt.title('AAM metric for different values of alpha')
plt.show()

## Evaluation of alpha sensitivity

Now that we have introduced the pipeline for computing the AAM error metric,
we apply the pipeline for different values of alpha. The results for the two cameras are shown below, the discussion of the results
can be found in the paper.

In [ ]:
from pipeline import pipeline
from math import floor

def make_table(alphas, aams):
    num_alphas = len(alphas)
    indices = (0, floor(num_alphas / 2.0), num_alphas-1)
    alphas_table = [alphas[ind] for ind in indices]
    aams_table = [aams[ind] for ind in indices]
    print('\\alpha \t & AAM \\\\')
    for alpha, aam in zip(alphas_table, aams_table):
        print('{:2.2f} \t & {:8.2f} \\\\'.format(alpha, aam))
    return

alphaList, AAM = pipeline(IMG_SET_ID = 4, scale=1e-2, num_colors=4, degree=5)
make_table(alphaList, AAM)

alphaList, AAM = pipeline(IMG_SET_ID = 7, scale=1e-2, num_colors=4, degree=5)
make_table(alphaList, AAM)

In [ ]:
alphaList, AAM = pipeline(IMG_SET_ID = 4, scale=1e-2, num_colors=3, degree=5)
make_table(alphaList, AAM)
alphaList, AAM = pipeline(IMG_SET_ID = 7, scale=1e-2, num_colors=3, degree=5)
make_table(alphaList, AAM)